In [20]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 222 kB in 1s (224 kB/s)
Reading package lists... Done


In [21]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [22]:
# Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [23]:
#Take a quick look at the data
spark.sparkContext.addFile(url)
home_sales_data = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")

# Show DataFrame
home_sales_data.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [24]:
# Create a temporary view of the DataFrame.
home_sales_data.createOrReplaceTempView('home_sales')


In [25]:
# Check out the temp view

spark.sql("SELECT * from home_sales limit 10").show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [26]:
# What is the average price for a four bedroom house sold in each year rounded to two decimal places?

spark.sql("""SELECT ROUND(AVG(price),2)
              AS Average_Price
              FROM home_sales
              WHERE bedrooms = 4""").show()

+-------------+
|Average_Price|
+-------------+
|    299661.01|
+-------------+



In [27]:
# What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""SELECT date_built AS Build_Year,
              ROUND(AVG(price),2) AS Average_Price
              FROM home_sales
              WHERE bedrooms = 3
              AND bathrooms = 3
              GROUP BY date_built""").show()


+----------+-------------+
|Build_Year|Average_Price|
+----------+-------------+
|      2015|     288770.3|
|      2013|    295962.27|
|      2014|    290852.27|
|      2012|    293683.19|
|      2016|    290555.07|
|      2010|    292859.62|
|      2011|    291117.47|
|      2017|    292676.79|
+----------+-------------+



In [28]:
# What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""SELECT date_built AS Build_Year,
              ROUND(AVG(price),2) AS Average_Price
              FROM home_sales
              WHERE bedrooms = 3
              AND bathrooms = 3
              AND floors = 2
              AND sqft_living >= 2000
              GROUP BY date_built""").show()



+----------+-------------+
|Build_Year|Average_Price|
+----------+-------------+
|      2015|    297609.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2012|    307539.97|
|      2016|     293965.1|
|      2010|    285010.22|
|      2011|    276553.81|
|      2017|    280317.58|
+----------+-------------+



In [29]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""SELECT view,
            ROUND(AVG(price),2)
            FROM home_sales
            WHERE price >= 350000
            GROUP BY view
            ORDER BY view""").show()



print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   0|           403848.51|
|   1|           401044.25|
|   2|           397389.25|
|   3|            398867.6|
|   4|           399631.89|
|   5|           401471.82|
|   6|           395655.38|
|   7|           403005.77|
|   8|           398592.71|
|   9|           401393.34|
|  10|           401868.43|
|  11|           399548.12|
|  12|           401501.32|
|  13|           398917.98|
|  14|           398570.03|
|  15|            404673.3|
|  16|           399586.53|
|  17|           398474.49|
|  18|           399332.91|
|  19|           398953.17|
+----+--------------------+
only showing top 20 rows

--- 0.867002010345459 seconds ---


In [30]:
# 7. Cache the the temporary table home_sales.
spark.sql("CACHE TABLE home_sales")

# Help from https://spark.apache.org/docs/latest/sql-ref-syntax-aux-cache-cache-table.html for this

DataFrame[]

In [31]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [32]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""SELECT view,
            ROUND(AVG(price),2)
            FROM home_sales
            WHERE price >= 350000
            GROUP BY view
            ORDER BY view""").show()



print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   0|           403848.51|
|   1|           401044.25|
|   2|           397389.25|
|   3|            398867.6|
|   4|           399631.89|
|   5|           401471.82|
|   6|           395655.38|
|   7|           403005.77|
|   8|           398592.71|
|   9|           401393.34|
|  10|           401868.43|
|  11|           399548.12|
|  12|           401501.32|
|  13|           398917.98|
|  14|           398570.03|
|  15|            404673.3|
|  16|           399586.53|
|  17|           398474.49|
|  18|           399332.91|
|  19|           398953.17|
+----+--------------------+
only showing top 20 rows

--- 0.47097063064575195 seconds ---


In [33]:
# Partition by the "date_built" field on the formatted parquet home sales data
home_sales_data.write.partitionBy("date_built").mode("overwrite").parquet("parquet_home_sales")

In [34]:
# 11. Read the parquet formatted data.
home_sales_parquet_df=spark.read.parquet('parquet_home_sales')

In [35]:
# 12. Create a temporary table for the parquet data.
home_sales_parquet_df.createOrReplaceTempView('parquet_home_sales_table')

In [36]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""SELECT view,
            ROUND(AVG(price),2)
            FROM parquet_home_sales_table
            WHERE price >= 350000
            GROUP BY view
            ORDER BY view""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|   0|           403848.51|
|   1|           401044.25|
|   2|           397389.25|
|   3|            398867.6|
|   4|           399631.89|
|   5|           401471.82|
|   6|           395655.38|
|   7|           403005.77|
|   8|           398592.71|
|   9|           401393.34|
|  10|           401868.43|
|  11|           399548.12|
|  12|           401501.32|
|  13|           398917.98|
|  14|           398570.03|
|  15|            404673.3|
|  16|           399586.53|
|  17|           398474.49|
|  18|           399332.91|
|  19|           398953.17|
+----+--------------------+
only showing top 20 rows

--- 0.8659226894378662 seconds ---


In [37]:
# Uncache the home_sales temporary table.
spark.sql("UNCACHE TABLE home_sales")

DataFrame[]

In [38]:
# Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False